In [ ]:
%pip install -qU kaggle

In [ ]:
from google.colab import files
files.upload()   # choose kaggle.json you downloaded from your Kaggle account page
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


Part 1. IEEE fraud detection

In [ ]:
COMP='ieee-fraud-detection'
!kaggle competitions download -c 'ieee-fraud-detection' -p /content/data


  0% 0.00/118M [00:00<?, ?B/s]
100% 118M/118M [00:00<00:00, 2.75GB/s]


In [ ]:
!unzip -q /content/data/*.zip -d /content/data

In [ ]:
!pip -q install -U autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.5 MB/s eta 0:00

In [ ]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import gc

directory = '/content/data/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models



In [ ]:
train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left') # Join id and transaction table, using TransactionID

del train_identity, train_transaction
gc.collect()

0

In [ ]:
# def downcast(df):
#     for c in df.select_dtypes(include=["float64"]).columns:
#         df[c] = pd.to_numeric(df[c], downcast="float")
#     for c in df.select_dtypes(include=["int64"]).columns:
#         df[c] = pd.to_numeric(df[c], downcast="integer")
#     for c in df.select_dtypes(include=["object"]).columns:
#         # cast to category if not too high-cardinality
#         if df[c].nunique(dropna=False) / len(df) < 0.5:
#             df[c] = df[c].astype("category")
#     return df

# train_data = downcast(train_data)

# UBSAMPLE FOR PROTOTYPING
# SUBFRAC = 0.25
# train_small = train_data.sample(frac=SUBFRAC, random_state=0)

# exclude heavy models
exclude = ["KNN", "NN_TORCH", "CAT"]  # KNN stores (most of) the training set in memory; neural nets & CatBoost can be heavy

# light hyperparameters
very_light_hp = {
    'GBM': [{
        'num_boost_round': 400,     # small model
        'learning_rate': 0.05,
        'num_leaves': 31,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'min_data_in_leaf': 100
    }],
    'LR': {}                        # tiny baseline, helps with many columns
}


In [ ]:

predictor = TabularPredictor(
    label=label,
    eval_metric=eval_metric,
    path=save_path,
    verbosity=3
).fit(
    train_data=train_data,
    presets='medium_quality',              # avoids auto-stacking, stays lean
    hyperparameters="light",
    excluded_model_types=exclude,
    num_bag_folds=0,                       # big RAM saver
    time_limit=3600
)

gc.collect()
# print("Prototype training finished on {:.0%} of data.".format(SUBFRAC))

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          8
GPU Count:          0
Memory Avail:       45.76 GB / 50.99 GB (89.7%)
Disk Space Avail:   178.36 GB / 225.83 GB (79.0%)
Presets specified: ['medium_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': False,
 'excluded_model_types': ['KNN', 'NN_TORCH', 'CAT'],
 'num_bag_folds': 0}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging':

[50]	valid_set's binary_logloss: 0.100743
[100]	valid_set's binary_logloss: 0.0924349
[150]	valid_set's binary_logloss: 0.0884887
[200]	valid_set's binary_logloss: 0.0856469
[250]	valid_set's binary_logloss: 0.0834745
[300]	valid_set's binary_logloss: 0.0818649
[350]	valid_set's binary_logloss: 0.0805544
[400]	valid_set's binary_logloss: 0.079232
[450]	valid_set's binary_logloss: 0.0784206
[500]	valid_set's binary_logloss: 0.0768095
[550]	valid_set's binary_logloss: 0.0759847
[600]	valid_set's binary_logloss: 0.075066
[650]	valid_set's binary_logloss: 0.0743114
[700]	valid_set's binary_logloss: 0.0736169
[750]	valid_set's binary_logloss: 0.0727746
[800]	valid_set's binary_logloss: 0.072003
[850]	valid_set's binary_logloss: 0.0714918
[900]	valid_set's binary_logloss: 0.0708359
[950]	valid_set's binary_logloss: 0.0705871
[1000]	valid_set's binary_logloss: 0.0700511
[1050]	valid_set's binary_logloss: 0.0695724
[1100]	valid_set's binary_logloss: 0.0685792
[1150]	valid_set's binary_logloss:

Saving /content/data/AutoGluonModels/models/LightGBMXT/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	0.9707	 = Validation score   (roc_auc)
	562.68s	 = Training   runtime
	2.44s	 = Validation runtime
	2421.0	 = Inference  throughput (rows/s | 5906 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Fitting model: LightGBM ... Training model for up to 414.77s of the 414.76s of remaining time.
	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 4
	Fitting with cpus=4, gpus=0, mem=9.7/42.0 GB
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05}


[50]	valid_set's binary_logloss: 0.0958781
[100]	valid_set's binary_logloss: 0.0878627
[150]	valid_set's binary_logloss: 0.0833765
[200]	valid_set's binary_logloss: 0.0809382
[250]	valid_set's binary_logloss: 0.0788046
[300]	valid_set's binary_logloss: 0.0760906
[350]	valid_set's binary_logloss: 0.0747061
[400]	valid_set's binary_logloss: 0.0730843
[450]	valid_set's binary_logloss: 0.071836
[500]	valid_set's binary_logloss: 0.0708809
[550]	valid_set's binary_logloss: 0.0699904
[600]	valid_set's binary_logloss: 0.0689386
[650]	valid_set's binary_logloss: 0.067958
[700]	valid_set's binary_logloss: 0.0671754
[750]	valid_set's binary_logloss: 0.0665508
[800]	valid_set's binary_logloss: 0.0650938
[850]	valid_set's binary_logloss: 0.0641821
[900]	valid_set's binary_logloss: 0.0632762
[950]	valid_set's binary_logloss: 0.0627643
[1000]	valid_set's binary_logloss: 0.0623129
[1050]	valid_set's binary_logloss: 0.061703
[1100]	valid_set's binary_logloss: 0.0612304
[1150]	valid_set's binary_logloss

	Ran out of time, early stopping on iteration 7112. Best iteration is:
	[6929]	valid_set's binary_logloss: 0.04548
Saving /content/data/AutoGluonModels/models/LightGBM/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/LightGBM/y_pred_proba_val.pkl
	0.9742	 = Validation score   (roc_auc)
	415.11s	 = Training   runtime
	1.29s	 = Validation runtime
	4573.5	 = Inference  throughput (rows/s | 5906 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Skipping RandomForestGini due to lack of time remaining.
Saving /content/data/AutoGluonModels/models/trainer.pkl
Skipping RandomForestEntr due to lack of time remaining.
Saving /content/data/AutoGluonModels/models/trainer.pkl
Skipping ExtraTreesGini due to lack of time remaining.
Saving /content/data/AutoGluonModels/models/trainer.pkl
Skipping ExtraTreesEntr due to lack of time remaining.
Saving /content/data/AutoGluonModels/models/trainer.pkl
Skipping NeuralNetFastAI due to lack of time remaining.
Saving /content/data/Aut

75

In [ ]:
predictor.save_space();

Loading: /content/data/AutoGluonModels/models/LightGBMXT/model.pkl
Saving /content/data/AutoGluonModels/models/LightGBMXT/model.pkl
Loading: /content/data/AutoGluonModels/models/LightGBM/model.pkl
Saving /content/data/AutoGluonModels/models/LightGBM/model.pkl
Loading: /content/data/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl
Saving /content/data/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl
Saving /content/data/AutoGluonModels/models/trainer.pkl


In [ ]:

results = predictor.fit_summary()

Loading: /content/data/AutoGluonModels/models/LightGBMXT/model.pkl
Loading: /content/data/AutoGluonModels/models/LightGBM/model.pkl
Loading: /content/data/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.974700     roc_auc       3.733004  977.845186                0.002188           0.051260            2       True          3
1             LightGBM   0.974212     roc_auc       1.291357  415.109457                1.291357         415.109457            1       True          2
2           LightGBMXT   0.970663     roc_auc       2.439460  562.684469                2.439460         562.684469            1       True          1
Number of models trained: 3
Types of models trained:
{'WeightedEnsembleModel', 'LGBModel'}
Bagging used: False 
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('category', []) :  31 | ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', ...]
('float', [])    : 395 | 

In [ ]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

del test_transaction, test_identity
gc.collect()

698

In [ ]:
# Find the missing columns in test_data compared to train_data
missing_cols = set(train_data.columns) - set(test_data.columns)
# Add the missing columns to test_data and fill with NaN
for c in missing_cols:
    test_data[c] = np.nan

# Ensure the order of columns in test_data is the same as in train_data, excluding the label column
test_data = test_data[train_data.drop(columns=[label]).columns]

In [ ]:
y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

Loading: /content/data/AutoGluonModels/models/LightGBM/model.pkl
Loading: /content/data/AutoGluonModels/models/LightGBMXT/model.pkl
Loading: /content/data/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


,0,1
0,0.999839,0.000161
1,0.999992,0.000008
2,0.999980,0.000020
3,0.999893,0.000107
4,0.999973,0.000027


In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba[1]
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)

In [ ]:
# larger_hp = {
#     'GBM': [{
#         'num_boost_round': 800,    # modest bump
#         'learning_rate': 0.04,
#         'num_leaves': 31,
#         'feature_fraction': 0.6,
#         'bagging_fraction': 0.8,
#         'min_data_in_leaf': 100
#     }],
#     'LR': {}
# }

# # Uncomment to run on FULL data (uses same exclusions; increase time_limit if you can):
# predictor_full = TabularPredictor(
#     label=label, eval_metric=eval_metric, path=save_path + "_full", verbosity=3
# ).fit(
#     train_data=train_data,
#     presets='medium_quality',
#     hyperparameters=larger_hp,
#     excluded_model_types=exclude,
#     num_bag_folds=0,
#     time_limit=900
# )
# print("Full-data training finished.")

In [ ]:

# predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3)
# predictor.fit(
#     train_data=traitrain_smalln_data,
#     presets="medium_quality",               # fast, no stacking
#     hyperparameters="light",                # lighter model configs
#     excluded_model_types=exclude,           # skip heavy models
#     num_bag_folds=0,                        # disable bagging to save RAM
#     time_limit=1000                          # give it a few minutes so something can train
# )